# explore dataset

In [5]:
import configparser
import logging
from ocs_sample_library_preview import *
import json
import pandas as pd
import pprint

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

config = configparser.ConfigParser()
config.read('config.ini')

ocsClient = OCSClient(config.get('Access', 'ApiVersion'), config.get('Access', 'Tenant'), config.get('Access', 'Resource'), 
                        config.get('Credentials', 'ClientId'), config.get('Credentials', 'ClientSecret'))

namespace_id = config.get('Configurations', 'Namespace')

In [3]:
# logger.setLevel(logging.DEBUG)
# logging.debug("tap tap")
# config.get('Access', 'ApiVersion')

In [7]:
# format the getSummaries output into a DataFrame based upon a stream query
def sds_summaries_format(query,start=None,end=None,property=None):
    
    if start == None or end == None:
        print("not implemented, specify start and end parameters")
        return None
    
    df_summaries = None
    df_summaries = pd.DataFrame(columns=['Count', 'Minimum', 'Maximum', 'Range', 'Total', 'Mean', 'StandardDeviation', 'PopulationStandardDeviation', 
                               'WeightedMean', 'WeightedStandardDeviation', 'WeightedPopulationStandardDeviation', 'Skewness', 'Kurtosis'])
    for stream in ocsClient.Streams.getStreams(namespace_id,query=query):
        try:
            summary = ocsClient.Streams.getSummaries(namespace_id,stream.Id,start=start,end=end,count=1,
                                                                value_class=None)
            #pprint.pprint(summary)
            for key,value in summary[0]['Summaries'].items():
                df_summaries.loc[stream.Name,key] = value[f'{property}']
        except Exception as e:
            print(f'getSummaries error: {str(e)}')
    
    return(df_summaries)

df = sds_summaries_format("nasa.bearing*",start="2004-02-12T10:52:39",end="2004-02-12T10:52:42",property="channel")
df

Count Minimum Maximum  Range    Total        Mean StandardDeviation  \
bearing1  20480    -0.4   0.503  0.903  -50.933 -0.00248696         0.0652627   
bearing4  20480  -0.334   0.354  0.688 -71.3801 -0.00348536         0.0488308   
bearing2  20480  -0.503   0.491  0.994 -34.7558 -0.00169706         0.0781181   
bearing3  20480  -0.786   0.933  1.719    -32.6  -0.0015918         0.0835223   

         PopulationStandardDeviation WeightedMean WeightedStandardDeviation  \
bearing1                   0.0652611  -0.00740165                 0.0382842   
bearing4                   0.0488296    0.0132388                 0.0309673   
bearing2                   0.0781162   -0.0505051                 0.0575789   
bearing3                   0.0835202    0.0952747                 0.0850878   

         WeightedPopulationStandardDeviation    Skewness   Kurtosis  
bearing1                           0.0382833   0.0215014   0.250411  
bearing4                           0.0309665    0.075476   0.148706  
bearing2                           0.0575775   0.0843913  0.0747565  
bearing3                           0.0850857  0.00732533    1.43493

In [8]:
# get a range of values using the first values timestamp as the start time
stream = "NASA.bearing1"
start_time = ocsClient.Streams.getFirstValue(namespace_id,stream)['timestamp']
pd.DataFrame.from_dict(ocsClient.Streams.getRangeValues(namespace_id,stream,None,start=start_time,skip=0,count=10,reverse=False,boundary_type=SdsBoundaryType.Exact))

channel  index                   timestamp
0    0.112    1.0        2004-02-12T10:52:39Z
1    0.049    2.0  2004-02-12T10:52:39.00005Z
2   -0.027    3.0   2004-02-12T10:52:39.0001Z
3   -0.110    4.0  2004-02-12T10:52:39.00015Z
4    0.007    5.0   2004-02-12T10:52:39.0002Z
5    0.122    6.0  2004-02-12T10:52:39.00025Z
6    0.134    7.0   2004-02-12T10:52:39.0003Z
7   -0.015    8.0  2004-02-12T10:52:39.00035Z
8   -0.168    9.0   2004-02-12T10:52:39.0004Z
9   -0.061   10.0  2004-02-12T10:52:39.00045Z

In [9]:
import datetime
start_datetime = (datetime.datetime.strptime(start_time,"%Y-%m-%dT%H:%M:%SZ"))
end_datetime = start_datetime + datetime.timedelta(hours=5)
df = pd.DataFrame.from_dict(ocsClient.Streams.getRangeValuesInterpolated(namespace_id,stream,None,start=start_time,end=end_datetime,count=10))
df

channel         index             timestamp
0  0.112000      1.000000  2004-02-12T10:52:39Z
1  0.062548  13677.005910  2004-02-12T11:25:59Z
2  0.080279   6839.002955  2004-02-12T11:59:19Z
3 -0.085000      1.000000  2004-02-12T12:32:39Z
4 -0.137623  13677.005910  2004-02-12T13:05:59Z
5 -0.000075   6839.002955  2004-02-12T13:39:19Z
6 -0.034000      1.000000  2004-02-12T14:12:39Z
7  0.056764  13677.005910  2004-02-12T14:45:59Z
8 -0.058663   6839.002955  2004-02-12T15:19:19Z
9 -0.059000      1.000000  2004-02-12T15:52:39Z

In [29]:
# format the getSummaries output into a DataFrame based upon a stream query
def sds_summaries_format(query,start=None,end=None,property=None):
    
    if start == None or end == None:
        print("not implemented, specify start and end parameters")
        return None
    
    df_summaries = None
    df_summaries = pd.DataFrame(columns=['Count', 'Minimum', 'Maximum', 'Range', 'Total', 'Mean', 'StandardDeviation', 'PopulationStandardDeviation', 
                               'WeightedMean', 'WeightedStandardDeviation', 'WeightedPopulationStandardDeviation', 'Skewness', 'Kurtosis'])
    for stream in ocsClient.Streams.getStreams(namespace_id,query=query):
        try:
            summary = ocsClient.Streams.getSummaries(namespace_id,stream.Id,start=start,end=end,count=1,
                                                                value_class=None)
            for key,value in summary[0]['Summaries'].items():
                df_summaries.loc[stream.Name,key] = value[f'{property}']
        except Exception as e:
            print(f'getSummaries error: {str(e)}')
    
    return(df_summaries)

df = sds_summaries_format("nasa.bearing*",start="2004-02-12T10:52:39",end="2004-02-12T10:52:42",property="channel")
df

Count Minimum Maximum  Range    Total        Mean StandardDeviation  \
bearing1  20480    -0.4   0.503  0.903  -50.933 -0.00248696         0.0652627   
bearing4  20480  -0.334   0.354  0.688 -71.3801 -0.00348536         0.0488308   
bearing2  20480  -0.503   0.491  0.994 -34.7558 -0.00169706         0.0781181   
bearing3  20480  -0.786   0.933  1.719    -32.6  -0.0015918         0.0835223   

         PopulationStandardDeviation WeightedMean WeightedStandardDeviation  \
bearing1                   0.0652611  -0.00740165                 0.0382842   
bearing4                   0.0488296    0.0132388                 0.0309673   
bearing2                   0.0781162   -0.0505051                 0.0575789   
bearing3                   0.0835202    0.0952747                 0.0850878   

         WeightedPopulationStandardDeviation    Skewness   Kurtosis  
bearing1                           0.0382833   0.0215014   0.250411  
bearing4                           0.0309665    0.075476   0.148706  
bearing2                           0.0575775   0.0843913  0.0747565  
bearing3                           0.0850857  0.00732533    1.43493